# TinyMS LeNet5 Server Tutorial

### In this tutorial, constructing a LeNet model, downloading dataset , training and and start the server of the model using TinyMS API will be demonstrated. 

## Prerequisite
 - Ubuntu: `18.04`
 - Python: `3.7.x`
 - Flask: `1.1.2`
 - MindSpore: `CPU-1.1.1`
 - TinyMS: `0.1.0`
 - numpy: `1.17.5`
 - opencv-python: `4.5.1.48`
 - Pillow: `8.1.0`
 - pip: `21.0.1`
 - requests: `2.18.4`
 
## Introduction

TinyMS is a high-level API which is designed for amateur of deep learning. It minimizes the number of actions of users required to construct, train, evaluate and serve a model. TinyMS also provides tutorials and documentations for developers. 

This tutorial consists of five parts, constructing the model, downloading dataset, training, define servable json and starting server, while sending a prediction request will be demonstrated in the `LeNet5_Client_tutorial.ipynb` file.

In [ ]:
import os
import json
import tinyms as ts
import tinyms.optimizers as opt
from tinyms.data import MnistDataset, download_dataset
from tinyms.vision import mnist_transform
from tinyms.model import Model, lenet5
from tinyms.serving import start_server
from tinyms.metrics import Accuracy
from tinyms.losses import SoftmaxCrossEntropyWithLogits

## 1. Construct the model

TinyMS encapsulates init and construct of the LeNet5 model, the line of the code is reduced to construct the LeNet5 model:

In [ ]:
net = lenet5(class_num=10)

## 2. Download dataset

The MNIST dataset will be downloaded if `mnist` folder didn't exist at the root. If `mnist` folder already exists, this step will not be performed.

In [ ]:
# download the dataset
mnist_path = '/root/mnist'
if not os.path.exists(mnist_path):
    ts.data.download_dataset('mnist', '/root')
    print('************Download complete*************')
else:
    print('************Dataset already exists.**************')

## 3. Train the model & evaluation

The dataset for both training and evaluation will be defined here, and the parameters for training also set in this block. A trained ckpt file will be saved to `/etc/tinyms/serving/lenet5` folder for later use, meanwhile the evaluation will be performed and the `Accuracy` can be checked

In [ ]:
# define the training and evaluation dataset
batch_size = 32
train_dataset = MnistDataset(os.path.join(mnist_path, "train"), shuffle=True)
train_dataset = mnist_transform.apply_ds(train_dataset)
eval_dataset = MnistDataset(os.path.join(mnist_path, "test"), shuffle=True)
eval_dataset = mnist_transform.apply_ds(eval_dataset)

# parameters for training
lr = 0.01
momentum = 0.9
epoch_size = 3
net_loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
net_opt = opt.Momentum(net.trainable_params(), lr, momentum)
net_metrics={"Accuracy": Accuracy()}

model = Model(net)
model.compile(loss_fn=net_loss, optimizer=net_opt, metrics=net_metrics)
print('************************Start training*************************')
model.train(epoch_size, train_dataset)
model.save_checkpoint('/etc/tinyms/serving/lenet5/lenet5.ckpt')
print('************************Finished training*************************')

model.load_checkpoint('/etc/tinyms/serving/lenet5/lenet5.ckpt')
print('************************Start evaluation*************************')
model.eval(eval_dataset)

## 4. Define servable.json

Define the lenet5 servable json file for model name, format and number of classes for serving. 

In [ ]:
servable_json = [{'name': 'lenet5', 
                  'description': 'This servable hosts a lenet5 model predicting numbers', 
                  'model': {
                      "name": "lenet5", 
                      "format": "ckpt", 
                      "class_num": 10}}]
os.chdir("/etc/tinyms/serving")
json_data = json.dumps(servable_json, indent=4)

with open('servable.json', 'w') as json_file:
    json_file.write(json_data)

## 5. Start server

### 5.1 Introduction
TinyMS Serving is a C/S(client/server) structure. TinyMS using [Flask](https://flask.palletsprojects.com/en/1.1.x/) whichi is a micro web framework written in python as the C/S communication tool. In order to serve a model, user must start server first. If successfully started, the server will listen to POST requests from 127.0.0.1 port 5000 sent by client and handle the requests using MindSpore backend which will construct the model, run the prediction and send the result back to the client.

### 5.2 Start server

run the following code block to start the server:

In [ ]:
start_server()

If you can see something similar to this:
```
* Serving Flask app "tinyms.serving.server.server" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```
that means you have successfully launched a server. Next, go to 'LeNet5_Client_tutorial.ipynb' to continue.

## Shutdown server

To restart server, clicke `Kernel` at the top, then click `Restart & Clear Output`

To shutdown server, if using terminal, simply CTRL + C to shutdown serving, if running in Jupyter, click `Kernel` at the top, then click `Shutdown`